In [ ]:
import pickle
import time
import matplotlib as plt
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, f_classif
from tensorflow import keras
drive.mount('/content/drive')

In [ ]:

class Client:
    def __init__(self, dataset_x, dataset_y, epoch_number, learning_rate,
                 weights, batch, features):
        self.dataset_x = dataset_x
        self.dataset_y = dataset_y
        self.epoch_number = epoch_number
        self.learning_rate = learning_rate
        self.weights = weights
        self.batch = batch
        self.features = features

    def train(self):
        model = keras.models.Sequential([
            keras.layers.Flatten(input_shape=[
                self.features,
            ]),
            keras.layers.Dense(200, activation='tanh'),
            keras.layers.Dense(100, activation='tanh'),
            keras.layers.Dense(12, activation='softmax')
        ])

        #setting weight of the model
        model.set_weights(self.weights)

        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])
        history = model.fit(self.dataset_x,
                            self.dataset_y,
                            epochs=self.epoch_number,
                            batch_size=self.batch)

        #getting the final_weight
        output_weight = model.get_weights()
        return output_weight

In [ ]:
file = open("/content/drive/MyDrive/Dataset/x_data", 'rb') x_data = pickle.load(file)
file = open("/content/drive/MyDrive/Dataset/y_data", 'rb') y_data = pickle.load(file)
file = open("/content/drive/MyDrive/Dataset/x_valid", 'rb') x_valid = pickle.load(file)
file = open("/content/drive/MyDrive/Dataset/y_valid", 'rb') y_valid = pickle.load(file)

In [ ]:
def feature_selection(x_data, y_data, x_valid):
    arr = [0] * 77
    fvalue_Best = SelectKBest(f_classif, k=35)
    x = np.array(x_data)
    y = np.array(y_data)

    for idx in range(len(x)):
        X_kbest = fvalue_Best.fit_transform(x[idx], y[idx])
        i, j = 0, 0
        while (j < 78 and i < 35):
            if (eval(str(X_kbest[0][i])) == eval(str(x[idx][0][j]))):
                arr[j] += 1
                i += 1
            j += 1

    temp_arr = []
    for idx in range(len(arr)):
        temp_arr.append((arr[idx], idx))
    temp_arr.sort()
    temp_arr.reverse()

    feature_count = 20
    final_ft = []
    for idx in range(len(temp_arr)):
        if idx < feature_count:
            final_ft.append(temp_arr[idx][1])
        else:
            if (temp_arr[idx][0] == temp_arr[idx - 1][0]):
                final_ft.append(temp_arr[idx][1])
            else:
                break
    final_ft.sort()

    # Selecting the features from x_data.
    x_data = np.array(x_data)
    x_data = x_data[:, :, final_ft]

    # Selecting the features from x_valid.
    x_valid = np.array(x_valid)
    x_valid = x_valid[:, final_ft]
    # print(final_ft)
    return x_data, x_valid, len(final_ft)

In [ ]:
def get_model(features):
    model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[
            features,
        ]),
        keras.layers.Dense(200, activation='tanh'),
        keras.layers.Dense(100, activation='tanh'),
        keras.layers.Dense(12, activation='softmax')
    ])

    return model

In [ ]:
# Function for averaging.
def model_average(client_weights):
    average_weight_list = []
    for index1 in range(len(client_weights[0])):
        layer_weights = []
        for index2 in range(len(client_weights)):
            weights = client_weights[index2][index1]
            layer_weights.append(weights)
        average_weight = np.mean(np.array([x for x in layer_weights]), axis=0)
        average_weight_list.append(average_weight)
    return average_weight_list

In [ ]:
def create_model(features):
  model = get_model(features)
  weight = model.get_weights()
  return weight

In [ ]:
def train_server(training_rounds, epoch, batch, learning_rate, x_data, x_valid,
                 features):
    # temp_variable
    # training_rounds=2
    # epoch=5
    # batch=128

    accuracy_list = []
    client_weight_for_sending = []

    for index1 in range(1, training_rounds):
        print('Training for round ', index1, 'started')
        client_weights_tobe_averaged = []
        for index in range(len(y_data)):
            print('-------Client-------', index)
            if index1 == 1:
                print(
                    'Sharing Initial Global Model with Random Weight Initialization'
                )
                initial_weight = create_model(features)
                client = Client(x_data[index], y_data[index], epoch,
                                learning_rate, initial_weight, batch, features)
                weight = client.train()
                client_weights_tobe_averaged.append(weight)
            else:
                client = Client(x_data[index], y_data[index], epoch,
                                learning_rate,
                                client_weight_for_sending[index1 - 2], batch,
                                features)
                weight = client.train()
                client_weights_tobe_averaged.append(weight)

        # Calculating the avearge weight from all the clients.
        client_average_weight = model_average(client_weights_tobe_averaged)
        client_weight_for_sending.append(client_average_weight)

        # Validating the model with avearge weight.
        model = get_model(features)

        model.set_weights(client_average_weight)
        model.compile(loss='sparse_categorical_crossentropy',
                      optimizer=keras.optimizers.SGD(lr=learning_rate),
                      metrics=['accuracy'])
        # print(x_valid)
        result = model.evaluate(x_valid, y_valid)
        accuracy = result[1]
        print('#######-----Acccuracy for round ', index1, 'is ', accuracy,
              ' ------########')
        accuracy_list.append(accuracy)
        accuracy_list_global.append(accuracy)

        # print("HEEEEEEEEEEEEEEEELLLLLLLLLLLOOOOOOOOOOOOOOO")
        for i in range(12):
            temp_y_valid = []
            temp_x_valid = []
            for ind, x in enumerate(y_valid):
                if x == i:
                    temp_y_valid.append(x)
                    temp_x_valid.append(list(x_valid[ind]))
            temp_y_valid = np.array(temp_y_valid)
            temp_x_valid = np.array(temp_x_valid)
            result = model.evaluate(temp_x_valid, temp_y_valid)
            accuracy = result[1]
            print('#######-----Acccuracy for attack ', i, 'is ', accuracy,
                  ' ------########')

    return accuracy_list, client_weight_for_sending

In [ ]:
def train_main(x_data, x_valid, features):
  start = time.time()
  training_accuracy, weights = train_server(100, 5, 64, 0.01, x_data, x_valid, features)
  end = time.time()
  print('TOTAL TIME = ', end - start)


In [ ]:
if __name__ == "__main__":
  x_data, x_valid, features = feature_selection(x_data, y_data, x_valid)
  # features = 77
  train_main(x_data, x_valid, features)
  pass
  print(accuracy_list_global)